In [ ]:
#Libraries
using Gen
using PyPlot
using Distributions
using LinearAlgebra
using Flux
using Random
using Distances
using JLD
using StatsBase
include("hmc_mod.jl")
include("helper_functions.jl")
include("rj_proposals.jl")
include("NUTS.jl");

In [ ]:
#---------------------------
#Load Boston Housing Dataset
#---------------------------
data = load("boston.jld")["boston"]

# Generating test/training sets:
nrow, ncol = size(data)
nrow_test  = div(nrow, 3)
nrow_train = nrow - nrow_test

x = data[:,1:13]
y = data[:,14]

dx = fit(UnitRangeTransform, x; dims=1, unit=true)
StatsBase.transform!(dx, x)
dy = fit(UnitRangeTransform, y; dims=1, unit=true)
StatsBase.transform!(dy, y);

x_raw = x
x = transpose(x)

#------------------------------------
#Hyperparameters and Helper Functions
#------------------------------------

#Select Network Goal
#network = "classifier"
network = "interpolator"

#Data hyperparameters
n = nrow #Number of samples per mode (classifier)
d = ncol-1 #Input dimension

#Network hyperparameters
k_real = 8 #Number of hidden nodes per layer
k_vector = [0.0 for i=1:k_real]
k_vector[k_real] = 1.0

#Layer hyperparameters
l_range = 4 #Maximum number of layers in the network
l_list = [Int(i) for i in 1:l_range]
l_real = 1

#NUTS
Δmax = 1000;

In [ ]:
#Bayesian Neural Net
function G(x, trace)
    activation = relu
    layers = trace[:l]
    ks = [trace[(:k,i)] for i=1:layers]
    for i=1:layers
        in_dim, out_dim = layer_unpacker(i, layers, ks)
        W = reshape(trace[(:W,i)], out_dim, in_dim)
        b = reshape(trace[(:b,i)], trace[(:k,i)])
        nn = Dense(W, b, activation)
        x = nn(x)
    end
    
    Wₒ = reshape(trace[(:W,layers+1)], 1, ks[layers])
    bₒ = reshape(trace[(:b,layers+1)], 1)
    
    nn_out = Dense(Wₒ, bₒ)
    return nn_out(x)
end;

@gen function interpolator(x::Array{Float64})
    
    #Create a blank choicemap
    obs = choicemap()::ChoiceMap
    
    #Draw number of layers
    l ~ categorical([1/length(l_list) for i=1:length(l_list)])
    l_real = l
    obs[:l] = l
    
    #Create individual weight and bias vectors
    #Loop through hidden layers
    k = [Int(0) for i=1:l+1]
    for i=1:l
        k[i] = @trace(categorical(k_vector), (:k,i))
        obs[(:k,i)] = k[i]
    end
    k[l+1] = @trace(categorical([1.0]), (:k,l+1))
    obs[(:k,l+1)] = k[l+1]
    
    α₁ = 0.001 #Gamma Scale for Hyperparameters
    α₂ = 0.1
    α₃ = 1.0
    αᵧ = 0.01
    
    ω₁ = 100
    ω₂ = k_real*100 #Neal (1996): Scaling relationship to # of hidden units
    ωᵧ = k_real*100
    τ₁ ~ gamma(ω₁,α₁) #Hidden Weights
    τ₂ ~ gamma(ω₁,α₂) #Hidden Biases
    τ₃ ~ gamma(ω₂,α₃) #Output Weights
    τᵧ ~ gamma(ωᵧ,α₃) #Noise Parameter for y
    #τ₄ ~ gamma() #Output Biases - Neal uses fixed sigmas here
    
    #Standard Deviations
    σ₁ = 1/τ₁
    σ₂ = 1/τ₂
    σ₃ = 1/τ₃
    σᵧ = 1/τᵧ
    
    #println(σᵧ)
    
    #Sample weight and parameter vectors
    W = [zeros(k[i]) for i=1:l+1]
    b = [zeros(k[i]) for i=1:l+1]
    for i=1:l+1
        if i == 1
            h = Int(d * k[i])
        else
            h = Int(k[i-1] * k[i])
        end

        if i<=l
            #Hidden Weights
            μ = zeros(h)
            if i == 1
                Σ = Diagonal([σ₁ for i=1:length(μ)])
            else
                Σ = Diagonal([σ₃ for i=1:length(μ)])
            end
            W[i] = @trace(mvnormal(μ,Σ), (:W,i))
            obs[(:W,i)] = W[i]
            
            #Hidden Biases
            μ2 = ones(k[i])
            Σ2 = Diagonal([σ₂ for i=1:length(μ2)])
            b[i] = @trace(mvnormal(μ2,Σ2), (:b,i))
            obs[(:b,i)] = b[i]
        else
            #Output Weights
            μₒ = zeros(k[l])
            Σₒ = Diagonal([σ₃ for i=1:length(μₒ)])
            W[i] = @trace(mvnormal(μₒ,Σₒ), (:W,i))
            obs[(:W,i)] = W[i]

            #Output Bias
            μ2ₒ = ones(1)
            Σ2ₒ = Diagonal([1.0 for i=1:length(μ2ₒ)])
            b[i] = @trace(mvnormal(μ2ₒ,Σ2ₒ), (:b,i))
            obs[(:b,i)] = b[i]
        end
    end
    
    #Return Network Scores for X
    scores = G(x,obs)
    scores = Flux.σ.(scores)
    
    #Regression Likelihood
    y = @trace(mvnormal(vec(scores), Diagonal([σᵧ for i=1:length(x[1,:])])), (:y))

    return scores
    
end;

#(best_trace,) = generate(interpolator, (x,), obs)
#println(best_trace[:τ₁])
#println(best_trace[:τ₂])
#println(best_trace[:τ₃])

test_scores = interpolator(x);

In [ ]:
#Register Observed Data - Bernoulli
obs_master = choicemap()::ChoiceMap
obs_master[:y] = y
obs = obs_master;

scores = []
mses = []
ks = []
best_traces = []
(best_trace,) = generate(interpolator, (x,), obs)
best_score = get_score(best_trace)
best_pred_y = Flux.σ.(G(x, best_trace))
best_mse = mse_regression(best_pred_y, y)

In [ ]:
#----------------
#Test Likelihood
#----------------
function likelihood(best_trace, best_mse, best_score)
    obs = obs_master;
    (trace,) = generate(interpolator, (x,), obs)
    
    pred_y = Flux.σ.(G(x, trace))
    mse = mse_regression(pred_y, y)
    score = get_score(trace)
    
    if mse > best_mse
        best_mse = mse
        best_score = score
        best_trace = trace
        best_pred_y = pred_y
    end
    push!(scores,score)
    push!(mses,mse)
    return(best_trace, best_mse, best_score)
end;

for i=1:10000
    best_trace, best_mse, best_score = likelihood(best_trace, best_mse, best_score)
end

PyPlot.scatter(mses, scores)
plt.title("Comparing Classifier Accuracy to Log Likelihood")
plt.xlabel("Classifier MSE")
plt.ylabel("Log Likelihood")
#plt.ylim(-100,1)
plt.legend()

In [ ]:
#--------------------
#RJMCMC - using NUTS
#--------------------
traces = []
scores = []
acc = []
acc_l = []
acc_w = []
l_results = []
epss = []

function within_move(trace, iters, obs, prev_trace)
    selection = select_selection(trace)
    (new_trace, hmc_score) = NUTS(trace, selection, false, obs, iters, iters, prev_trace)
    #println(hmc_score)
    if rand(Uniform(0,1)) < exp(hmc_score)
        trace = new_trace
        accepted = 1.0
        println("Accepted")
    else
        trace = prev_trace
        accepted = 0.0
        #println("Not Accepted")
    end
    #push!(traces, trace)
    push!(acc, accepted)
    push!(acc_w, accepted)
    #accepted && println("Within accepted")
    return trace
end

function layer_move(trace, iters, obs, prev_trace)
    
    #Determine birth or death
    current_l = trace[:l]
    
    if current_l == last(l_list)
        move_type = 0
    elseif current_l == l_list[1]
        move_type = 1
    else
        move_type = bernoulli(0.5)
    end
    move = "Empty"
    
    obs_master = choicemap()::ChoiceMap
    obs_master[:y] = y

    #HMC Move 1
    selection = select_selection(trace)
    hmc1_trace = trace
    (hmc1_trace, hmc1_score) = NUTS(hmc1_trace, selection, false, obs, iters, iters, prev_trace)

    #RJ Move
    if move_type == 1
        move = "Birth"
        rj_trace = layer_birth(hmc1_trace)
    else
        move = "Death"
        rj_trace = layer_death(hmc1_trace)
    end

    #HMC Move 2
    hmc2_trace = rj_trace
    (hmc2_trace, hmc2_score) = NUTS(hmc2_trace, selection, false, obs, iters, iters, hmc2_trace)

    score1 = get_score(prev_trace)
    score2 = get_score(hmc2_trace)
    logscore = (score2 - score1)
    score = exp(logscore) #+ hmc1_score - hmc2_score)
    #println("$move: $score")
    
    if rand(Uniform(0,1)) < score
        accepted = true
        trace = hmc2_trace
        println("New ks accepted! Current ks: $trace[:l]")
    else
        println("Sticking with the old l!")
        accepted = false
        trace = prev_trace
    end

    #println("$move Old Trace: $score1; Pre-HMC: $score_test; Post-HMC: $score2")
        
    push!(acc, accepted)
    push!(acc_l, accepted)
    push!(scores, score)
    return trace
end


function rjmcmc(starting_trace, iters)
    trace = starting_trace
    l = trace[:l]
    ks = [trace[(:k,i)] for i=1:trace[:l]]
    println("Beginning RJMCMC")
    println("Starting ks: $ks")
    println("--------------------------------")

    for i=1:iters
        l = trace[:l]
        obs = obs_master;
        if i%10 == 0
            #println("Epoch $i Acceptance Prob: $(sum(acc)/length(acc))")
            #println("Epoch $i layer count: $l, ks: $ks")
            println("Epoch $i Within Acceptance Prob: $(sum(acc_w)/length(acc_w))")
            #println("Epoch $i Layer Acceptance Prob: $(sum(acc_l)/length(acc_l))")
            #println([trace[(:k,i)] for i=1:trace[:l]])
        end
        
        #Gibbs sampling for hyperparameters
        prev_trace = trace
        trace, obs = select_hyperparameters(prev_trace, obs)
        
        #Indicator variable for move type
        u = rand(Uniform(0,1))
        if u > 1.0
            (trace) = layer_move(trace, 10, obs, prev_trace)
        else
            (trace) = within_move(trace, 100, obs, prev_trace)
        end
        push!(traces, trace)
        push!(scores, get_score(trace))
        push!(l_results, trace[:l])
    end
    println("Finished")
end
 
runs = 2

obs_master = choicemap()::ChoiceMap
obs_master[:y] = y
obs = obs_master;
(starting_trace,) = generate(interpolator, (x,), obs)
#starting_trace = best_trace

include("NUTS.jl");
rjmcmc(starting_trace,runs);